In [1]:
import retrieval


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tfidf = retrieval.TF_IDFSearch() # 리트리버 정의
tfidf.get_sparse_embedding() # 위키 기반의 tfidfvectorizer을 만들고 passge_embedding을 만듭니다.
tfidf.build_faiss() # faiss를 통해 빠른 서치가 가능하도록 faiss index를 만듭니다.
result = tfidf.search_query() # faiss를 사용하지 않고 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.
result2 = tfidf.search_query_faiss() # faiss를 사용하여 tfidf를 통해 test dataset의 질문과 관련된 문서를 찾습니다.
result # result와 result2 는 같은 모양새로 반환됩니다.

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (1133 > 512). Running this sequence through the model will result in indexing errors


56737
./retrieval_result 폴더가 없어서 만듭니다.
Passage embedding을 만듭니다.
p_embedding.shape : (56737, 50000)
임베딩을 피클형태로 저장했습니다.
Faiss indexer을 만듭니다.
Faiss Indexer을 저장했습니다.


Sparse retrieval: 100%|██████████| 600/600 [00:00<00:00, 20630.77it/s]


쿼리당 20개의 문서를 faiss indexer을 통해 찾습니다.


Sparse retrieval: 100%|██████████| 600/600 [00:00<00:00, 23413.99it/s]


DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})

In [2]:
#bm25 test
import importlib
import retrieval
importlib.reload(retrieval)

# BM25Search 인스턴스 생성
bm25 = retrieval.BM25Search()

# BM25 임베딩 생성 및 저장/로드
bm25.get_sparse_embedding()

# 쿼리 검색 수행
result = bm25.search_query_bm25()

# 결과 출력
result

Data route: /data/ephemeral/data/train_dataset/
Test data route: /data/ephemeral/data/test_dataset/
Wiki route: /data/ephemeral/data/wikipedia_documents.json
Data path: ./bm25_retrieval_result


Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


File loaded successfully
위키 컨텍스트의 수: 56737
./bm25_retrieval_result 폴더가 없어서 만듭니다.
Passage embedding을 만듭니다.
임베딩을 피클 형태로 저장했습니다.


BM25 retrieval: 100%|██████████| 600/600 [00:00<00:00, 13833.61it/s]


DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
})

In [ ]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.train() #model.train()은 기존에 주어진 데이터셋을 활용하여 훈련합니다. model.train(train_dataset, eval_dataset)으로 본인이 직접 지정한 데이터를 훈련할 수도 있습니다.
# 이때, train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
# 모델은 Extraction_based_MRC_outputs + model_name의 폴더로 저장됩니다.
# 모델은 학습이 끝나면 가장 낮은 validaiton loss를 기준으로 저장됩니다.
model.inference(result)
# retrieval을 통해 얻어낸 결과를 predict_reuslt에 json파일 형태로 저장합니다.
"""inference는 반드시
DatasetDict({
    validation: Dataset({
        features: ['context', 'id', 'question'],
        num_rows: 600
    })
}) 의 형태를 따라야 합니다.
"""



In [ ]:
from MRC import Extraction_based_MRC
model = Extraction_based_MRC()
model.load_model() # 체크포인트의 제일 마지막 폴더를 따라가서 trainer_state.json을 열고 거기서 best_model_checkpoint의 경로를 통해 모델을 불러옵니다.
# 불러온 모델은 inference할 수 있습니다.

In [ ]:
import All_dataset
from arguments import Extraction_based_MRC_arguments, Dense_search_retrieval_arguments, Generation_based_MRC_arguments
datas = All_dataset.prepare_dataset(Extraction_based_MRC_arguments)
train = datas.get_mrc_train_dataset()
eval = datas.get_mrc_eval_dataset()
# test =  datas.get_mrc_test_dataset(리트리버결과)
print(train)
print(eval)

# datas = All_dataset.prepare_dataset(Dense_search_retrieval_arguments)
# train = datas.get_dense_train_dataset()
# eval = datas.get_dense_valid_dataset()

# datas = All_dataset.prepare_dataset(Generation_based_MRC_arguments)
# train = datas.get_generative_MRC_train_dataset()
# eval = datas.get_generative_MRC_valid_dataset()

"""
이처럼 All_dataset.prepare_dataset(argument에서 불러온 모델의 arg)를 통해 각 모델에서 쓰이는 데이터들을 볼 수 있습니다.
아무것도 주지 않으면 기본으로 주어진 데이터를 바탕으로 데이터를 전처리합니다.
get_mrc_train_dataset(여러분이 바꾼 데이터) 를 통해 바꾼 데이터를 전처리할 수도 있습니다.
train 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['train'], eval 데이터셋의 형태는 load_from_disk(기존에 주어진 데이터경로)['validation']의 형태여야 합니다.
"""

